En este ejercicio, vamos a recuperar la información correspondiente a la página "https://cronicasperiodisticas.wordpress.com/", específicamente, cada uno de las entradas que corresponde a los cronistas y sus crónicas allí incluidos. 

El resultado de este ejercicio será un directorio general donde se guardarán los archivos de cada cronista, y un directorio para cada cronista en el cual se guardarán los textos en formato txt. 

La lógica que utilizaremos para recuperar la información será ir desde el artículo más reciente (https://cronicasperiodisticas.wordpress.com/2017/04/08/stop-running/) hasta el primero.

Navegaremos por el sitio a través de las fechas del final de cada entrada. De cada entrada recuperamos la categoría (que es el nombre del cronista), el título y el texto principal. Con la url podremos obtener la clave para el nombre del archivo (ej: Uncategorized/stop-running.txt)

In [4]:

from bs4 import BeautifulSoup
import requests

In [1]:
# El enlace a la primera entrada con una crónica (punto de inicio)

url = "https://cronicasperiodisticas.wordpress.com/2017/03/31/la-herencia-de-juan-manuel-fangio/"

In [ ]:
r = requests.get(url)
r

In [ ]:
contenido =  r.content
print(contenido[:1000])

In [5]:
sopa = BeautifulSoup(contenido, 'html.parser')

Después de estos pasos, ya tenemos listo el contenido de la página. Ahora, vamos a identificar cómo extraer el título y el texto principal.

El título se encuentra en la etiqueta \<h2\> y tiene la clase "pagetitle". Así, podemos recuperar el título de la página. Probemos esta idea a continuación:

In [ ]:
titulo = sopa.find_all("h2", class_="pagetitle")[0].get_text()
print(titulo)

¡Éxito!

Lo que hicimos en este punto fue:

1. Buscar todas las coincidencias de `<h2 class="pagetitle">` en la "sopa"
2. Aunque el resultado es solamente uno, debemos sacarlo de la lista. Para eso simplemente indicamos que nos recupere el primer resultado `[0]`.
3. Finalmente, extraemos solamente el texto del título con la función `get_text()`

El siguiente paso consistirá en recuperar el texto de la entrada.

Al inspeccionar la entrada, vemos que el contenido de la entrada se encuentra en el `<div>` con la clase `"entry"`. Así que haremos lo siguiente:

In [ ]:
contenido = [parrafo.text for parrafo in texto.select("p") for texto in sopa.find_all("div", class_="entry")]
print(contenido)

Nuevamente, explico lo que hicimos en esta línea de código:

1. Iteramos a través de todos los elementos que contiene el `div` con la clase `"entry"`.
2. Dentro de cada uno de ellos seleccionamos a los que corresponden a párrafos (`p`), así evitamos que nos regrese otra información que no deseamos.
3. Extraemos el texto de cada párrafo.
4. Todo esto queda guardado en una lista en la variable contenido.

Solamente nos queda por recuperar el nombre del cronista, que está marcado como categoría. Para este caso, está dentro de una etiqueta `<a>` y no tiene una clase sino un `rel="category tag`. En este caso la recuperación de la información es un poco diferente:

In [ ]:
nombre_autor = [autor for autor in sopa.find_all('a', {"rel": "category tag"})[0]]
print(nombre_autor[0])

En este caso, lo que hicimos fue aprovechar el atributo `rel` para seleccionar la etiqueta `a` correspondiente. La manera de hacerlo es a través de un 'diccionario' `{"rel": "category tag"}`

Facil ¿no es así?

Ya tenemos entonces nuestros tres elementos principales:

* Título
* Nombre del autor
* Texto de la crónica

Ahora, vamos a revisar cómo podremos navegar crónica por crónica, de la más nueva a la más antigua.

Para ello, vamos a necesitar extraer la url del div `<div class="leftnav">` donde se encuentra la etiqueta `a` con el atributo `rel="prev"`.

In [ ]:
nav = [e['href'] for e in sopa.find_all("a", {"rel": "prev"})]
print(nav[0])

Ya con esta información, podemos iniciar la construcción de los archivos. 

En primer lugar, demos acceso al Google Drive para que nuestros archivos se guarden allí.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Ahora, vamos a convertir nuestros scripts en funciones para poder usarlo en el loop más fácilmente y hacer nuestro código más legible y sostenible.

In [16]:
import os
import requests
from bs4 import BeautifulSoup

def sopa(url):
  '''
  Con esta función construimos una sopa de cada url que necesitemos
  '''
  r = requests.get(url)
  c = r.content
  return BeautifulSoup(c, 'html.parser')

def titulo(url):
  '''
  Esta función regresa el título
  '''
  return sopa(url).find_all("h2", class_="pagetitle")[0].get_text()

def nombre_autor(url):
  '''
  Esta función regresa el nombre del autor
  '''
  nom = [autor for autor in sopa(url).find_all('a', {"rel": "category tag"})[0]]
  return nom[0]

def contenido(url):
  '''
  Con esta función recuperamos el contenido de cada crónica
  '''
  parrafos = []
  for texto in sopa(url).find_all("div", class_="entry"):
    for parrafo in texto.select("p"):
      parrafos.append(parrafo.text)
  return parrafos


def cronica(url):
  '''
  Esta es la función central que vincula todo.
  Esta construye cada archivo de texto
  '''
  path = "/content/drive/MyDrive/Colab Notebooks"
  autor = nombre_autor(url)
  os.makedirs(os.path.join(path, autor), exist_ok=True)
  titul = titulo(url)
  filename = os.path.join(path, autor, titul.lower().replace(" ", "-").replace(r"\xa0", "-").replace(":", "").replace("/", "")+'.txt')
  try:
    pathlib.Path.exists(filename)
    print("{} ya existe".format(titul))
  except:
    with open(filename, 'w+', encoding='utf-8') as f:
      f.write(f"{autor}\n")
      f.write(f"{titul}\n")
      for c in contenido(url):
        f.write(f"{c}\n")
    print("{} ha sido creado".format(titul))
    f.close()

Y después de tener nuestras funciones, es momento de ponerlas a correr por todo el sitio.

Para ello tenemos que usar un while loop,

In [ ]:
url = "https://cronicasperiodisticas.wordpress.com/2017/03/31/la-herencia-de-juan-manuel-fangio/"

nav = [e['href'] for e in sopa(url).find_all("a", {"rel": "prev"})]
cronica(url)


while nav:
  url = nav[0]
  cronica(url)
  nav = [e['href'] for e in sopa(url).find_all("a", {"rel": "prev"})]

  
